In [1]:
! apt install firefox  xvfb > /dev/null
! pip3 install  pyvirtualdisplay selenium webdriver_manager  > /dev/null

In [2]:
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [191]:
#column_names = ["Brand","Model","Ex-showroom","RTO","Insurance","Kits","Total","Speed","CC","Power","Torque","C-ratio","Bore","Stroke","Cylinders","Valves","Cooling","Fuel","capacity","Mileage","Weight","Height","Length","Clearance","Wheelbase","ABS","Rear-brake","Front-brake","Rear-suspension","Front-suspension","Brake-pads","Slipper-clutch","QuickShifter","Warranty","Service-Network"]
#class_names = ["o-dOKno o-bNCMFw o-cJrNdO","o-dOKno o-bNCMFw o-cJrNdO","notavailable","notavailable","notavailable","notavailable","o-Hyyko o-bPYcRG o-eqqVmt","o-cJrNdO","o-cJrNdO"]
common_url = 'https://www.bikewale.com/'
data_class = 'USCqPI'
bike_class = 'o-dOKno o-bNCMFw o-cJrNdO'.replace(" ", ".")
name_class = 'o-zmksK lI0JO_ o-cpnuEd'.replace(" ", ".")
value_class = 'mLrdoz lI0JO_'.replace(" ", ".")
price_class = 'o-Hyyko o-bPYcRG o-eqqVmt'.replace(" ", ".")
#bikes = ["ktm-bikes/rc-390/","ktm-bikes/duke-390/","yamaha-bikes/r15/","suzuki-bikes/gixxer-sf-250/","ktm-bikes/rc-200/","ktm-bikes/rc-125/"]
collapsed_sections = ["(//div[@class='USCqPI']/div[4]/ul/li[1]/div/div[3])","(//div[@class='USCqPI']/div[4]/ul/li[2]/div/div[3])","(//div[@class='USCqPI']/div[4]/ul/li[3]/div/div[3])","(//div[@class='USCqPI']/div[5]/ul/li/div/div[2])"]
bikes = ["tvs-bikes/apache-rr-310/","bmw-bikes/g-310-r/","zontes-bikes/350r/","ktm-bikes/rc-390/","ktm-bikes/duke-390/","yamaha-bikes/r15/","suzuki-bikes/gixxer-sf-250/","ktm-bikes/rc-200/","ktm-bikes/rc-125/","ktm-bikes/duke-200/","ktm-bikes/duke-125/","royalenfield-bikes/hunter-350/","yamaha-bikes/mt-15/","keeway-bikes/k300-r/","yezdi-bikes/adventure/","yezdi-bikes/roadster/","yezdi-bikes/scrambler/","jawa-bikes/42-bobber/","jawa-bikes/42-fj/","jawa-bikes/perak/","jawa-bikes/350/","jawa-bikes/42/","triumph-bikes/speed-400/","triumph-bikes/scrambler-400/","triumph-bikes/speed-t4/","harleydavidson-bikes/x440/","husqvarna-bikes/vitpilen-250/","husqvarna-bikes/svartpilen-401/","suzuki-bikes/v-strom-sx/","honda-bikes/cb300r/","honda-bikes/cb350rs/","honda-bikes/cb350/","tvs-bikes/apache-rtr-310/"]
found = False
try:
  results = pd.read_csv("bike_data.csv", index_col=False)
  found = True
except:
  results = pd.DataFrame()

In [192]:
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
browser = webdriver.Firefox(options=options)

In [193]:
def click_collapsed_div(browser,xpath):
  try:
    elem = WebDriverWait(browser, 2000).until(EC.presence_of_element_located((By.XPATH, xpath)))
    try:
      browser.implicitly_wait(2)
      elem.click()
      #print(f"Clicked on element: {elem}")
    except Exception as e:
      print(f"Error clicking element: {elem}: {e}")
  except Exception as e:
    print(f"Error finding element")

def get_bike_details_selenium(url):
  print(f"fetching URL: {url}")
  browser.get(url)
  for collapsed_section in collapsed_sections:
    click_collapsed_div(browser,collapsed_section)

  bike_details = {}

  bike_name = browser.find_element(By.CLASS_NAME, bike_class)
  bike_details["URL"] = url
  bike_details["Brand"] = bike_name.text.strip().split(" ")[0]
  bike_details["Model"] = bike_name.text.strip()[len(bike_details["Brand"]):]
  bike_price = browser.find_element(By.CLASS_NAME, price_class)
  bike_details["Price"] = bike_price.text.strip()[2:]
  print("Expanded collapsed divs")
  print("Getting bike details")
  element_names = browser.find_elements(By.CLASS_NAME, name_class)
  element_values = browser.find_elements(By.CLASS_NAME, value_class)
  #element_values = browser.find_elements(By.XPATH, f"(//div[@class='{value_class}']/div)")

  for i in range(len(element_names)):
    bike_details[element_names[i].get_attribute("textContent")] = element_values[i].get_attribute("textContent")

  # for i in range(len(column_names)):
  #   try:
  #     element = browser.find_element(By.CLASS_NAME, class_names[i].replace(" ", "."))
  #     if element:
  #       bike_details[column_names[i]] = element.text.strip()
  #     else:
  #       bike_details[column_names[i]] = np.nan
  #   except:
  #     pass

  return bike_details

def find_and_export_bike_data(bikes,results):
  print("Looking for your bikes!")
  for bike in bikes:
    if "bikewale" in bike:
      url = bike
    else:
      url = common_url + bike
    if found and url in results['URL'].values:
      print(f"Skipping {bike} as it had already been fetched earlier.")
    else:
      bike_details = get_bike_details_selenium(url)
      bike_data = pd.DataFrame([bike_details]).reset_index(drop=True)
      results = pd.concat([results, bike_data], ignore_index=True)
      print(f"Fetched data for {bike}")

  print("Done! Exporting to csv")
  results.to_csv("bike_data.csv", index=False)
  return results


find_and_export_bike_data(bikes,results)

Looking for your bikes!
fetching URL: https://www.bikewale.com/tvs-bikes/apache-rr-310/
Expanded collapsed divs
Getting bike details
Fetched data for tvs-bikes/apache-rr-310/
fetching URL: https://www.bikewale.com/bmw-bikes/g-310-r/
Expanded collapsed divs
Getting bike details
Fetched data for bmw-bikes/g-310-r/
fetching URL: https://www.bikewale.com/zontes-bikes/350r/
Expanded collapsed divs
Getting bike details
Fetched data for zontes-bikes/350r/
Skipping ktm-bikes/rc-390/ as it had already been fetched earlier.
Skipping ktm-bikes/duke-390/ as it had already been fetched earlier.
Skipping yamaha-bikes/r15/ as it had already been fetched earlier.
Skipping suzuki-bikes/gixxer-sf-250/ as it had already been fetched earlier.
Skipping ktm-bikes/rc-200/ as it had already been fetched earlier.
Skipping ktm-bikes/rc-125/ as it had already been fetched earlier.
fetching URL: https://www.bikewale.com/ktm-bikes/duke-200/
Expanded collapsed divs
Getting bike details
Fetched data for ktm-bikes/du

,URL,Brand,Model,Price,Displacement,Max Power,Max Torque,Mileage - ARAI,Mileage - Owner Reported,Riding Range,...,Start Type,Kill Switch,Pillion Backrest,Pillion Grab Rail,Pillion Seat,Pillion Footrest,Front Suspension Preload Adjuster,Rear Suspension Preload Adjuster,Additional Features,4th Service
0,https://www.bikewale.com/ktm-bikes/rc-390/,KTM,RC 390,"3,21,382",373.27 cc,42.9 bhp @ 9000 rpm,37 Nm @ 7000 rpm,29 kmpl,29 kmpl,397.3 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,Yes,Yes,Quickshifter +,NaN
1,https://www.bikewale.com/ktm-bikes/duke-390/,KTM,390 Duke,"2,97,784",398.63 cc,45.3 bhp @ 8500 rpm,39 Nm @ 6500 rpm,NaN,30 kmpl,450 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,Yes,Yes,"Track Screen, Ride-by-wire, Quickshifter+",NaN
2,https://www.bikewale.com/yamaha-bikes/r15/,Yamaha,R15 V4,"1,83,465",155 cc,18.1 bhp @ 10000 rpm,14.2 Nm @ 7500 rpm,51.4 kmpl,45 kmpl,495 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,TFT Display,13000 Kms/390 Days
3,https://www.bikewale.com/suzuki-bikes/gixxer-s...,Suzuki,Gixxer SF 250,"1,94,450",249 cc,26.13 bhp @ 9300 rpm,22.2 Nm @ 7300 rpm,35 kmpl,36 kmpl,432 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,Clip-On Handle Bars,NaN
4,https://www.bikewale.com/ktm-bikes/rc-200/,KTM,RC 200,"2,20,228",199.5 cc,24.6 bhp @ 10000 rpm,19.2 Nm @ 8000 rpm,43.5 kmpl,35 kmpl,479.5 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,Fully Adjustable Handlebar,NaN
5,https://www.bikewale.com/ktm-bikes/rc-125/,KTM,RC 125,"1,91,631",124.7 cc,14.34 bhp @ 9250 rpm,12 Nm @ 8000 rpm,NaN,36 kmpl,493.2 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,Fully Adjustable Handlebar,NaN
6,https://www.bikewale.com/tvs-bikes/apache-rr-310/,TVS,Apache RR 310,"2,75,000",312.2 cc,37.48 bhp @ 9800 rpm,29 Nm @ 7900 rpm,34.7 kmpl,32 kmpl,352 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,"5"" TFT Screen Connected Cluster, RT-DSC, TPMS,...",NaN
7,https://www.bikewale.com/bmw-bikes/g-310-r/,BMW,G 310 R,"2,90,000",313 cc,33.52 bhp @ 9250 rpm,28 Nm @ 7500 rpm,30 kmpl,31 kmpl,341 km,...,Electric Start,Yes,No,Yes,Yes,Yes,No,Yes,Electric throttle Ride by Wire,NaN
8,https://www.bikewale.com/zontes-bikes/350r/,Zontes,350R,"2,79,000",348 cc,38.2 bhp @ 9500 rpm,32.8 Nm @ 7500 rpm,NaN,-,-,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,TFT Screen,NaN
9,https://www.bikewale.com/ktm-bikes/duke-200/,KTM,200 Duke,"1,99,100",199.5 cc,24.67 bhp @ 10000 rpm,19.3 Nm @ 8000 rpm,NaN,34 kmpl,455.6 km,...,Electric Start,Yes,No,Yes,Stepped Seat,Yes,No,Yes,5-inch TFT Display,NaN


In [137]:
browser.get(common_url + bikes[0])
for collapsed_section in collapsed_sections:
    click_collapsed_div(browser,collapsed_section)

Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="8ba9b4b6-67f0-46f7-8324-4ac14228facc")>
Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="88d62935-6edc-471b-804d-3ab77e2ca5e8")>
Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="bf5bf6af-4805-465a-920d-1f9662257546")>
Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="55a353e3-a279-4ec8-a84d-c16418dc1af7")>


In [110]:
element_names = browser.find_elements(By.CLASS_NAME, name_class.replace(" ", "."))
element_values = browser.find_elements(By.CLASS_NAME, value_class.replace(" ", "."))

In [147]:
value_class = ".mLrdoz.lI0JO_"
name_class = ".o-zmksK.lI0JO_.o-cpnuEd"
xpath = "(//*[@class='"+value_class+"'])"
res = browser.find_elements(By.CSS_SELECTOR, value_class)
names = browser.find_elements(By.CSS_SELECTOR, name_class)
for i in range(len(res)):
  print(names[i].get_attribute("textContent") +" : "+res[i].get_attribute("textContent"))

Displacement : 155 cc
Max Power : 18.1 bhp @ 10000 rpm 
Max Torque : 14.2 Nm @ 7500 rpm 
Mileage - ARAI : 51.4 kmpl
Mileage - Owner Reported : 45 kmpl
Riding Range : 495 km
Top Speed : 140 kmph
Riding Modes : Track and Street 
Transmission : 6 Speed Manual 
Gear Shifting Pattern : 1 Down 5 Up 
Cylinders : 1 
Bore : 58 mm
Stroke : 58.7 mm
Valves Per Cylinder : 4 
Compression Ratio : 11.6 : 1 
Spark Plugs : 1 Per Cylinder
Cooling System : Liquid Cooled 
Clutch : Assist And Slipper Clutch 
Fuel Tank Capacity : 11 litres
Reserve Fuel Capacity : 1.76 litres
Emission Standard : BS6 Phase 2 
Fuel Type : Petrol 
Front Suspension : Upside Down Front Forks 
Rear Suspension : Linked-Type Monocross Suspension 
Braking System : Dual Channel ABS 
Front Brake Type : Disc 
Front Brake Size : 282 mm
Caliper - Front : 2 Piston
Rear Brake Type : Disc 
Rear Brake Size : 220 mm
Caliper - Rear : 1 Piston
Wheel Type : Alloy 
Front Wheel Size : 17 inch
Rear Wheel Size : 17 inch
Front Tyre Size : 100/80 - 17 
